In [1]:
# analyze new york crime data
inFilePath="file:///home/prabir/Work/apachespark/datasets/NYPD_7_Major_Felony_Incidents.csv"

In [2]:
sc

<SparkContext master=local[2] appName=PySparkShell>

In [3]:
fileRdd=sc.textFile(inFilePath)

In [4]:
fileRdd.count()

1123466

In [6]:
fileRdd.take(2)

['OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1',
 '1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"']

In [7]:
fileRdd.take(1)

['OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1']

In [5]:
header=fileRdd.first()
print(header)
print(type(header))

OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1
<class 'str'>


In [6]:
datawoheader=fileRdd.filter(lambda x:x!=header)

In [7]:
datawoheader.take(10)

['1,f070032d,09/06/1940 07:30:00 PM,Friday,Sep,6,1940,19,9,7,2010,BURGLARY,FELONY,D,66,BROOKLYN,N.Y. POLICE DEPT,987478,166141,"(40.6227027620001, -73.9883732929999)"',
 '2,c6245d4d,12/14/1968 12:20:00 AM,Saturday,Dec,14,1968,0,12,14,2008,GRAND LARCENY,FELONY,G,28,MANHATTAN,N.Y. POLICE DEPT,996470,232106,"(40.8037530600001, -73.955861904)"',
 '3,716dbc6f,10/30/1970 03:30:00 PM,Friday,Oct,30,1970,15,10,31,2008,BURGLARY,FELONY,H,84,BROOKLYN,N.Y. POLICE DEPT,986508,190249,"(40.688874254, -73.9918594329999)"',
 '4,638cd7b7,07/18/1972 11:00:00 PM,Tuesday,Jul,18,1972,23,7,19,2012,GRAND LARCENY OF MOTOR VEHICLE,FELONY,F,73,BROOKLYN,N.Y. POLICE DEPT,1005876,182440,"(40.6674141890001, -73.9220463899999)"',
 '5,6e410287,05/21/1987 12:01:00 AM,Thursday,May,21,1987,0,5,28,2009,GRAND LARCENY,FELONY,K,75,BROOKLYN,N.Y. POLICE DEPT,1017958,182266,"(40.6668988440001, -73.878495425)"',
 '6,7eebfe3c,02/01/1990 09:00:00 AM,Thursday,Feb,1,1990,9,9,17,2014,GRAND LARCENY,FELONY,K,105,QUEENS,N.Y. POLICE DEPT,

In [21]:
type(datawoheader)

pyspark.rdd.PipelinedRDD

In [14]:
from collections import namedtuple
from io import StringIO
import csv

In [26]:
header

'OBJECTID,Identifier,Occurrence Date,Day of Week,Occurrence Month,Occurrence Day,Occurrence Year,Occurrence Hour,CompStat Month,CompStat Day,CompStat Year,Offense,Offense Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location 1'

In [9]:
fields=header.replace(" ","_")

In [10]:
fields


'OBJECTID,Identifier,Occurrence_Date,Day_of_Week,Occurrence_Month,Occurrence_Day,Occurrence_Year,Occurrence_Hour,CompStat_Month,CompStat_Day,CompStat_Year,Offense,Offense_Classification,Sector,Precinct,Borough,Jurisdiction,XCoordinate,YCoordinate,Location_1'

In [18]:
crimeclass=namedtuple('CrimeClass',fields)

In [19]:
def parser(row):
    strtlist=csv.reader(StringIO(row))
    strtlist=next(strtlist)
    return crimeclass(*strtlist)


In [20]:
crime=datawoheader.map(lambda x:parser(x))

In [21]:
crime.take(1)

[CrimeClass(OBJECTID='1', Identifier='f070032d', Occurrence_Date='09/06/1940 07:30:00 PM', Day_of_Week='Friday', Occurrence_Month='Sep', Occurrence_Day='6', Occurrence_Year='1940', Occurrence_Hour='19', CompStat_Month='9', CompStat_Day='7', CompStat_Year='2010', Offense='BURGLARY', Offense_Classification='FELONY', Sector='D', Precinct='66', Borough='BROOKLYN', Jurisdiction='N.Y. POLICE DEPT', XCoordinate='987478', YCoordinate='166141', Location_1='(40.6227027620001, -73.9883732929999)')]

In [64]:
crime.first()

CrimeClass(OBJECTID='1', Identifier='f070032d', Occurrence_Date='09/06/1940 07:30:00 PM', Day_of_Week='Friday', Occurrence_Month='Sep', Occurrence_Day='6', Occurrence_Year='1940', Occurrence_Hour='19', CompStat_Month='9', CompStat_Day='7', CompStat_Year='2010', Offense='BURGLARY', Offense_Classification='FELONY', Sector='D', Precinct='66', Borough='BROOKLYN', Jurisdiction='N.Y. POLICE DEPT', XCoordinate='987478', YCoordinate='166141', Location_1='(40.6227027620001, -73.9883732929999)')

In [22]:
crime.first().Offense

'BURGLARY'

In [23]:
crime.map(lambda x: x.Offense).countByValue()

defaultdict(int,
            {'BURGLARY': 191369,
             'FELONY ASSAULT': 184042,
             'GRAND LARCENY': 428993,
             'GRAND LARCENY OF MOTOR VEHICLE': 101963,
             'MURDER & NON-NEGL. MANSLAUGHTE': 4574,
             'NA': 1,
             'RAPE': 13779,
             'ROBBERY': 198744})

In [24]:
crime.map(lambda x: x.Occurrence_Year).countByValue()

defaultdict(int,
            {'': 244,
             '1905': 2,
             '1908': 1,
             '1910': 3,
             '1911': 1,
             '1912': 1,
             '1913': 4,
             '1914': 2,
             '1915': 3,
             '1920': 1,
             '1940': 1,
             '1945': 2,
             '1946': 1,
             '1950': 1,
             '1954': 1,
             '1955': 1,
             '1956': 1,
             '1958': 1,
             '1959': 1,
             '1960': 1,
             '1964': 1,
             '1965': 2,
             '1966': 7,
             '1968': 1,
             '1969': 1,
             '1970': 2,
             '1971': 1,
             '1972': 2,
             '1973': 5,
             '1974': 3,
             '1975': 2,
             '1976': 2,
             '1977': 3,
             '1978': 2,
             '1979': 6,
             '1980': 5,
             '1981': 1,
             '1982': 5,
             '1983': 1,
             '1984': 4,
             '1985': 8,
 

In [25]:
crimenewrdd=crime.filter(lambda x: (x.Offense!='NA' and x.Occurrence_Year!=''))

In [26]:
crimenewrdd.take(10)

[CrimeClass(OBJECTID='1', Identifier='f070032d', Occurrence_Date='09/06/1940 07:30:00 PM', Day_of_Week='Friday', Occurrence_Month='Sep', Occurrence_Day='6', Occurrence_Year='1940', Occurrence_Hour='19', CompStat_Month='9', CompStat_Day='7', CompStat_Year='2010', Offense='BURGLARY', Offense_Classification='FELONY', Sector='D', Precinct='66', Borough='BROOKLYN', Jurisdiction='N.Y. POLICE DEPT', XCoordinate='987478', YCoordinate='166141', Location_1='(40.6227027620001, -73.9883732929999)'),
 CrimeClass(OBJECTID='2', Identifier='c6245d4d', Occurrence_Date='12/14/1968 12:20:00 AM', Day_of_Week='Saturday', Occurrence_Month='Dec', Occurrence_Day='14', Occurrence_Year='1968', Occurrence_Hour='0', CompStat_Month='12', CompStat_Day='14', CompStat_Year='2008', Offense='GRAND LARCENY', Offense_Classification='FELONY', Sector='G', Precinct='28', Borough='MANHATTAN', Jurisdiction='N.Y. POLICE DEPT', XCoordinate='996470', YCoordinate='232106', Location_1='(40.8037530600001, -73.955861904)'),
 CrimeCl

In [27]:
def cordextract(value):
    value=value[1:-2]
    lat,long=map(float,value.split(","))
    return (lat,long)

In [28]:
crimenewrdd.map(lambda x:cordextract(x.Location_1))

PythonRDD[12] at RDD at PythonRDD.scala:53

In [39]:
crimenewrdd.take(1)

[CrimeClass(OBJECTID='1', Identifier='f070032d', Occurrence_Date='09/06/1940 07:30:00 PM', Day_of_Week='Friday', Occurrence_Month='Sep', Occurrence_Day='6', Occurrence_Year='1940', Occurrence_Hour='19', CompStat_Month='9', CompStat_Day='7', CompStat_Year='2010', Offense='BURGLARY', Offense_Classification='FELONY', Sector='D', Precinct='66', Borough='BROOKLYN', Jurisdiction='N.Y. POLICE DEPT', XCoordinate='987478', YCoordinate='166141', Location_1='(40.6227027620001, -73.9883732929999)')]

In [46]:
cfd=crimenewrdd.filter(lambda x:cordextract(x.Location_1)[0]>=40.00 and
                      cordextract(x.Location_1)[0]<=40.50           and
                      cordextract(x.Location_1)[1]>=-74.259         and
                      cordextract(x.Location_1)[1]<=-74.00)

In [47]:
cfd.count()

16

In [35]:
import gmplot

In [48]:
gmap = gmplot.GoogleMapPlotter(37.428, -122.145, 16)


In [56]:
lats=cfd.filter(lambda x: x.Offense=='RAPE').map(lambda x: cordextract(x.Location_1)[0]).collect()

In [57]:
lats

[]

In [58]:
long=cfd.filter(lambda x: x.Offense=='RAPE').map(lambda x: cordextract(x.Location_1)[1]).collect()

In [54]:
gmap.scatter(lats, long, '#DE1515', size=40, marker=False)

In [55]:
gmap.draw("BURGLARY_NY.html")

In [59]:
long

[]